In [1]:
import urllib2
from bs4 import *
from urlparse import urljoin
import MySQLdb as mysql
import jieba
import re
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')   
ignorewords=set(['the','of','to','and','a','in','is','it'])
class crawler:
    def __init__(self,dbname):
        self.con=mysql.connect(host="localhost",user="root",passwd="root",db=dbname,charset='utf8')
        self.cursor=self.con.cursor()
    def __del__(self):
        self.cursor.close()
        self.con.close()
    def dbcommit(self):
        self.con.commit()
    def createindextables(self):
        self.cursor.execute("CREATE TABLE IF NOT EXISTS `link` (`rowid` decimal(20,0) NOT NULL DEFAULT '0',`fromid` decimal(20,0) DEFAULT NULL,`toid` decimal(20,0) DEFAULT NULL,PRIMARY KEY (`rowid`),KEY `urltoidx` (`toid`), KEY `urlfromidx` (`fromid`)) ENGINE=InnoDB DEFAULT CHARSET=utf8")
        self.cursor.execute("CREATE TABLE IF NOT EXISTS `linkwords` (`wordid` decimal(20,0) NOT NULL DEFAULT '0', `linkid` decimal(20,0) NOT NULL DEFAULT '0', PRIMARY KEY (`wordid`,`linkid`) ) ENGINE=InnoDB DEFAULT CHARSET=utf8")
        self.cursor.execute("CREATE TABLE IF NOT EXISTS `wordlist` (`word` varchar(50) DEFAULT NULL, `rowid` bigint(20) NOT NULL AUTO_INCREMENT, PRIMARY KEY (`rowid`), KEY `wordidx` (`word`) ) ENGINE=InnoDB DEFAULT CHARSET=utf8")
        self.cursor.execute("CREATE TABLE IF NOT EXISTS `urllist` (`url` varchar(200) DEFAULT NULL, `rowid` bigint(20) NOT NULL AUTO_INCREMENT, PRIMARY KEY (`rowid`), KEY `urlidx` (`url`) ) ENGINE=InnoDB DEFAULT CHARSET=utf8")
        self.cursor.execute("CREATE TABLE IF NOT EXISTS `wordlocation` (`location` varchar(50) NOT NULL DEFAULT '', `wordid` decimal(20,0) NOT NULL DEFAULT '0', `urlid` decimal(20,0) NOT NULL DEFAULT '0', PRIMARY KEY (`wordid`,`urlid`,`location`), KEY `wordurlidx` (`wordid`) ) ENGINE=InnoDB DEFAULT CHARSET=utf8")
        self.dbcommit()
    # Index an individual page
    def addtoindex(self,url,soup):
        if self.isindexed(url): return
        print 'Indexing '+url
        # Get the individual words
        text=self.gettextonly(soup)
        words=self.separatewords(text)
        # Get the URL id
        urlid=self.getentryid('urllist','url',url)
        # Link each word to this url
        for i in range(len(words)):
            word=words[i]
            if word in ignorewords: continue
            if len(word)>50: continue
            wordid=self.getentryid('wordlist','word',word)
            self.cursor.execute("insert into wordlocation(urlid,wordid,location) values (%d,%d,%d)" % (urlid,wordid,i))
    def isindexed(self,url):
        #there are some wrong
        u=self.cursor.execute("select rowid from urllist where url='%s'" % url)
        if u>0:
            # Check if it has actually been crawled
            v=self.cursor.execute('select * from wordlocation where urlid=%d' % self.cursor.fetchone()[0])
            if v>0: return True
        return False
    def getentryid(self,table,field,value,createnew=True):
        cur=self.cursor.execute("select rowid from %s where %s='%s'" % (table,field,value))
        if cur==0:
            self.cursor.execute("insert into %s (%s) values ('%s')" % (table,field,value))
            return self.getentryid(table,field,value)
        else:
            return self.cursor.fetchone()[0]
    def gettextonly(self,soup):
        v=soup.string
        if v==None:
            c=soup.contents
            resulttext=''
            for t in c:
                subtext=self.gettextonly(t)
                resulttext+=subtext+'\n'
            return resulttext
        else:
            return v.strip( )
    def separatewords(self,text):
        return list(jieba.cut_for_search(re.sub(u'([^\u4e00-\u9fa5]+)',"",text.decode('utf8'))))
    def addlinkref(self,urlFrom,urlTo,linkText):
        pass
    def crawl(self,pages,depth=1):
        for i in range(depth):
            newpages=set()
            for page in pages:
                try:
                    c=urllib2.urlopen(page)
                except:
                    print "Could not open %s" % page
                    continue
                soup=BeautifulSoup(c.read( ),"lxml")
                self.addtoindex(page,soup)
                links=soup('a')
                for link in links:
                    if('href' in dict(link.attrs)):
                        url=urljoin(page,link['href'])
                        if url.find("'")!=-1: continue	
                        url=url.split('#')[0] # remove location portion
                        if url[0:4]=='http' and not self.isindexed(url):
                            newpages.add(url)
                        linkText=self.gettextonly(link)
                        self.addlinkref(page,url,linkText)
                self.dbcommit( )
            pages=newpages
        

In [2]:
mycrawler=crawler('searchEngine')
mycrawler.createindextables()
mycrawler.dbcommit()
mycrawler.crawl(['http://www.gowhich.com/blog/147?utm_source=tuicool&utm_medium=referral'])

In [4]:
print mycrawler.cursor.execute('select * from wordlocation where wordid=13')
print [row for row in mycrawler.cursor.fetchall()]

61
[(u'100', 13L, 1L), (u'1001', 13L, 1L), (u'1059', 13L, 1L), (u'1063', 13L, 1L), (u'1067', 13L, 1L), (u'1068', 13L, 1L), (u'1082', 13L, 1L), (u'1088', 13L, 1L), (u'1105', 13L, 1L), (u'12', 13L, 1L), (u'127', 13L, 1L), (u'156', 13L, 1L), (u'163', 13L, 1L), (u'165', 13L, 1L), (u'178', 13L, 1L), (u'191', 13L, 1L), (u'207', 13L, 1L), (u'221', 13L, 1L), (u'241', 13L, 1L), (u'257', 13L, 1L), (u'271', 13L, 1L), (u'279', 13L, 1L), (u'28', 13L, 1L), (u'291', 13L, 1L), (u'298', 13L, 1L), (u'307', 13L, 1L), (u'461', 13L, 1L), (u'468', 13L, 1L), (u'484', 13L, 1L), (u'49', 13L, 1L), (u'493', 13L, 1L), (u'551', 13L, 1L), (u'560', 13L, 1L), (u'574', 13L, 1L), (u'583', 13L, 1L), (u'595', 13L, 1L), (u'613', 13L, 1L), (u'636', 13L, 1L), (u'643', 13L, 1L), (u'663', 13L, 1L), (u'668', 13L, 1L), (u'709', 13L, 1L), (u'714', 13L, 1L), (u'751', 13L, 1L), (u'757', 13L, 1L), (u'765', 13L, 1L), (u'77', 13L, 1L), (u'772', 13L, 1L), (u'779', 13L, 1L), (u'790', 13L, 1L), (u'80', 13L, 1L), (u'802', 13L, 1L), (u'82